In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(style='dark')
import plotly.express as px

from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.tree import export_graphviz
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.datasets import load_iris
from sklearn import tree
from sklearn.cluster import KMeans

df = pd.read_csv('/kaggle/input/top-spotify-songs-from-20102019-by-year/top10s.csv', encoding='ISO-8859-1', index_col = 0)

df = df.drop_duplicates()

df.head()

df.shape


In [ ]:
# Correlation Heatmap
plt.figure(figsize=(10,8))
sns.heatmap(df.corr(), annot=True,cmap = sns.cm.rocket_r)

cont_df = pd.crosstab(df['top genre'],df['artist'])
cont_df
cont_mat = cont_df.to_numpy()
cont_mat.sum()
cont_mat.sum(axis=0)
cont_mat.sum(axis=1)
expected_mat = np.outer(cont_mat.sum(axis=1)/cont_mat.sum(),cont_mat.sum(axis=0))
chi2_values = (cont_mat - expected_mat)**2 / expected_mat
chi2_values.sum()

from scipy import stats
1 - stats.chi2.cdf(chi2_values.sum(),8967)

In [ ]:
#genres_piechart=plt.pie(df['top genre'].value_counts().iloc[:5],explode=[0.1,0,0,0,0] ,labels=df['top genre'].value_counts().iloc[:5].index,
#autopct='%1.1f%%', shadow=True, startangle=50)

We see that 'Pop' genres dominated last decade in Spotify.

'Dance pop' is the most prevalent genre in the last decade. 'Pop' and 'Canadian pop' are second and third, respectively.

In [ ]:
#genres_piechart=plt.pie(df['artist'].value_counts().iloc[:5],explode=[0.1,0,0,0,0] ,labels=df['artist'].value_counts().iloc[:5].index,
#autopct='%1.1f%%', shadow=True, startangle=50)

In [ ]:
le = LabelEncoder()
for column in df.columns.values:
  if df[column].dtypes == 'object':
    le.fit(df[column].values)
    df[column] = le.transform(df[column])
df.head()

feature_cols = ['top genre','year','bpm','dnce','dB','live','val','dur','acous','spch']

X = df[feature_cols] # Features
y = df['pop'] # Target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training and 30% test
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)
y_pred
y_test.values
np.sqrt(np.sum(np.square(y_pred-y_test)))/y_test.shape[0]

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression(fit_intercept=True)

feature_cols = ['top genre','year','bpm','dnce','dB','live','val','dur','acous','spch']

X = df[feature_cols] # Features
y = df['pop'] # Target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training and 30%
model.fit(X_train,y_train)
pred = model.predict(X_test)

print(np.sqrt(np.sum(np.square(pred-y_test)))/y_test.shape[0])




In [ ]:

sns.regplot(x=X,y=df.pop).set_title('Valence (Positive mood) vs Energy',fontsize=15);
plt.xlabel('Valence (Positive mood)',fontsize=12);
plt.ylabel('Energy',fontsize=12);

In [ ]:
plt.scatter(range(0,pred.shape[0]),pred-y_test)
#plt.scatter(range(0,pred.shape[0]),pred)
#plt.scatter(range(0,pred.shape[0]),y_test)
plt.plot(range(0,pred.shape[0]),[0]*pred.shape[0])

In [ ]:



#feature_cols = ['bpm','nrgy','dnce','dB','live','val','dur','acous','spch']
#values=df[feature_cols]
#X_train, X_test, y_train, y_test = train_test_split(values, y, test_size=0.3, random_state=1) # 70% training and 30% test

#wcss = []    
#for k in range(1, 11):
    #kmeans = KMeans(n_clusters=k, max_iter=1000, n_init = 10, random_state = 0).fit(values)
    #wcss.append(kmeans.inertia_)
    
#Plotting the results onto a line graph, allowing us to observe 'The elbow'
#plt.figure(dpi=100)
#plt.plot(range(1, 11), wcss)
#plt.title('The elbow method')
#plt.xlabel('Number of clusters')
#plt.ylabel('WCSS') #within cluster sum of squares
#plt.tight_layout()




In [ ]:
pop_norm=((df['pop']-df['pop'].mean())/df['pop'].std()).to_numpy()
spch_norm=((df['spch']-df['spch'].mean())/df['spch'].std()).to_numpy()
acous_norm=((df['acous']-df['acous'].mean())/df['acous'].std()).to_numpy()
dur_norm=((df['dur']-df['dur'].mean())/df['dur'].std()).to_numpy()
val_norm=((df['val']-df['val'].mean())/df['val'].std()).to_numpy()
live_norm=((df['live']-df['live'].mean())/df['live'].std()).to_numpy()
dB_norm=((df['dB']-df['dB'].mean())/df['dB'].std()).to_numpy()
dnce_norm=((df['dnce']-df['dnce'].mean())/df['dnce'].std()).to_numpy()
nrgy_norm=((df['nrgy']-df['nrgy'].mean())/df['nrgy'].std()).to_numpy()
bpm_norm=((df['bpm']-df['bpm'].mean())/df['bpm'].std()).to_numpy()
values=np.array([spch_norm,acous_norm,dur_norm,val_norm,live_norm,dB_norm,dnce_norm,nrgy_norm,bpm_norm])

feature_cols = [bpm_norm,nrgy_norm,dnce_norm,live_norm,val_norm,dur_norm,acous_norm,spch_norm]

import scipy.cluster.hierarchy as sch

X_linkage=sch.linkage(feature_cols, method = 'ward')
plt.figure(figsize=(8,4),dpi=150)
dendrogrm = sch.dendrogram(X_linkage)
plt.title('Dendrogram')
plt.xlabel('pop')
plt.ylabel('Euclidean distance')
plt.tight_layout()
#plt.savefig("Dendrogram_example_1.png")
#files.download("Dendrogram_example_1.png")



In [ ]:

#feature_cols = ['bpm','nrgy','dnce','dB','live','val','dur','acous','spch']


values=df[feature_cols]


kmeans = KMeans(n_clusters=8, max_iter=1000).fit(values)

df["clusters"] = kmeans.labels_
df.groupby(by=['clusters']).mean()


df.head()

In [ ]:
#sns.catplot(x="clusters", y="pop", data=df)


In [ ]:
ax = sns.violinplot(x="clusters", y="pop",
                    data=df, palette="Set2", split=True,
                    scale="count")

In [ ]:
(np.mean([df['pop'][df['clusters'] == 0]]),
np.mean([df['pop'][df['clusters'] == 1]]),
np.mean([df['pop'][df['clusters'] == 2]]),
np.mean([df['pop'][df['clusters'] == 3]]),
np.mean([df['pop'][df['clusters'] == 4]]),
np.mean([df['pop'][df['clusters'] == 5]]),
np.mean([df['pop'][df['clusters'] == 6]]),
np.mean([df['pop'][df['clusters'] == 7]]))


In [ ]:
(np.mean([df['bpm'][df['clusters'] == 5]]),
np.mean([df['nrgy'][df['clusters'] == 5]]),
np.mean([df['dnce'][df['clusters'] == 5]]),
np.mean([df['dB'][df['clusters'] == 5]]),
np.mean([df['live'][df['clusters'] ==5]]),
np.mean([df['val'][df['clusters'] == 5]]),
np.mean([df['dur'][df['clusters'] ==5]]),
np.mean([df['acous'][df['clusters'] == 5]]),
np.mean([df['spch'][df['clusters'] ==5]]),)

